# Loading the ECCOv4 native model grid parameters 

## Objectives

Briefly show how to open the ECCOv4 grid file as an `xarray` dataset, plot parameters, and create subsets of the grid dataset.

## Introduction

The ECCOv4 model grid parameters are provided as a single NetCDF file. It can be downloaded using the `ecco_podaac_download` function as described in the [download tutorial](https://ecco-v4-python-tutorial.readthedocs.io/Downloading_ECCO_Datasets_from_PODAAC_Python.html#ECCO_download-module). The ShortName for the dataset is **ECCO_L4_GEOMETRY_LLC0090GRID_V4R4**. The grid parameters file has no time dimension, but `ecco_podaac_download` requires a StartDate and EndDate to be specified; any date in the range 1992-2017 can be used.

## Load the ECCOv4 model grid parameter NetCDF file

Because the ECCOv4 model grid parameter data is provided in a single file you can use the ``open_dataset`` routine from ``xarray`` to open it. 

First set up your environment.

In [ ]:
import numpy as np
import xarray as xr
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## Import the ecco_v4_py library into Python
## =========================================

## -- If ecco_v4_py is not installed in your local Python library, 
##    tell Python where to find it.  For example, if your ecco_v4_py
##    files are in /Users/ifenty/ECCOv4-py/ecco_v4_py, then use:

from os.path import expanduser,join
import sys
user_home_dir = expanduser('~')
sys.path.append(join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

In [ ]:
## Set top-level file directory for the ECCO NetCDF files

## change ECCO_dir as needed
ECCO_dir = join(user_home_dir,'Downloads','ECCO_V4r4_PODAAC')

In [ ]:
## open ECCO datasets needed for the tutorial (without loading them into memory yet)

ShortNames_list = ["ECCO_L4_GEOMETRY_LLC0090GRID_V4R4"]
if incloud_access == True:
    from ecco_s3_retrieve import ecco_podaac_s3_get_diskaware
    files_nested_list = ecco_podaac_s3_get_diskaware(ShortNames=ShortNames_list,\
                                                     StartDate='1992-01',EndDate='2017-12',\
                                                     max_avail_frac=0.5,\
                                                     download_root_dir=ECCO_dir)
    grid = xr.open_mfdataset(files_nested_list[ShortNames_list[0]])
else:
    grid_params_file = "GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc"
    grid = xr.open_mfdataset(join(ECCO_dir,ShortNames_list[0],grid_params_file))

In [ ]:
# show contents of grid_dataset
grid

### Plot grid parameters using plot_tiles function

Let's plot two of the model grid parameter fields ``hFacC`` (tracer cell thickness factor) and ``rA`` (grid cell surface area)

First we plot ``hFac``:

In [ ]:
ecco.plot_tiles(grid.hFacC.isel(k=0), cmap='gray', show_colorbar=True,);

In [ ]:
ecco.plot_tiles(grid.rA, cmap='jet', show_colorbar=True);
plt.suptitle('Model grid cell surface area [m^2]')

plt.show()

### Plot subset of global domain

Once the file has been "opened" using ``open_dataset``, we can use `Dataset.isel` to subset the file if we don't want to plot the full global domain.

In [ ]:
grid_subset = grid.isel(tile=[1,10,12],k=[0,1,2,3])
grid_subset

Notice that ``grid_subset`` only has 3 tiles (1, 10, 12) and 4 depth levels (0, 1, 2, 3), as expected. The subset indices can also be passed to `isel` as a Python dictionary, with the same result.

In [ ]:
subset_ind = {'tile':[1,10,12],'k':[0,1,2,3]}

grid_subset = grid.isel(subset_ind)
grid_subset

Let's plot ``hFacC`` and ``rA`` again

In [ ]:
ecco.plot_tiles(grid_subset.hFacC.isel(k=0), cmap='gray', show_colorbar=True,);
'Model grid cell surface area [m^2] in tiles 1, 10, and 12 '

Notice that 10 of the 13 tiles are blank because they were not loaded.

In [ ]:
ecco.plot_tiles(grid_subset.rA, cmap='jet', show_colorbar=True);
'Model grid cell surface area [m^2]'